In [1]:
pip install tensorflow-recommenders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text
import numpy as np
import pandas as pd

2024-06-15 14:27:52.983800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 14:27:52.984031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 14:27:53.169158: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_excel("/kaggle/input/ide-bisnis-2/ide_bisnis-_4_.xlsx", sheet_name='mirip')
df.head()

,user_id,hobi,pekerjaan,lama pengalaman,aset,ide_bisnis,label
0,1,memotret,guru,1 Sampai 3 Tahun,rendah,Jasa Fotografi Sekolah,5
1,1,memotret,guru,1 Sampai 3 Tahun,rendah,Fotografi Personal untuk Acara Keluarga,5
2,1,memotret,guru,1 Sampai 3 Tahun,rendah,Kelas Fotografi untuk Anak-anak,4
3,1,memotret,guru,1 Sampai 3 Tahun,rendah,Jasa Fotografi Online,3
4,1,memotret,guru,1 Sampai 3 Tahun,rendah,Buku Panduan Fotografi untuk Pemula,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          106 non-null    int64 
 1   hobi             106 non-null    object
 2   pekerjaan        106 non-null    object
 3   lama pengalaman  106 non-null    object
 4   aset             106 non-null    object
 5   ide_bisnis       106 non-null    object
 6   label            106 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 5.9+ KB


In [5]:
df.nunique()

user_id            21
hobi               10
pekerjaan          15
lama pengalaman     7
aset                3
ide_bisnis         97
label               5
dtype: int64

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Reshape the column to fit the scaler's expected input and scale it
df['label'] = scaler.fit_transform(df[['label']])
print(df)

     user_id      hobi  pekerjaan   lama pengalaman      aset  \
0          1  memotret       guru  1 Sampai 3 Tahun    rendah   
1          1  memotret       guru  1 Sampai 3 Tahun    rendah   
2          1  memotret       guru  1 Sampai 3 Tahun    rendah   
3          1  memotret       guru  1 Sampai 3 Tahun    rendah   
4          1  memotret       guru  1 Sampai 3 Tahun    rendah   
..       ...       ...        ...               ...       ...   
101       21  memotret  pengacara  3 Sampai 5 Tahun  menengah   
102       21  memotret  pengacara  3 Sampai 5 Tahun  menengah   
103       21  memotret  pengacara  3 Sampai 5 Tahun  menengah   
104       21  memotret  pengacara  3 Sampai 5 Tahun  menengah   
105       21  memotret  pengacara  3 Sampai 5 Tahun  menengah   

                                   ide_bisnis  label  
0                      Jasa Fotografi Sekolah   1.00  
1    Fotografi Personal untuk Acara Keluarga    1.00  
2             Kelas Fotografi untuk Anak-anak   0.75  

In [7]:
ide_bisnis = tf.data.Dataset.from_tensor_slices(dict(df[['hobi', 'pekerjaan', 'lama pengalaman', 'aset', 'ide_bisnis', 'label']]))

ide_bisnis = ide_bisnis.map(lambda x: {'hobi' : x['hobi'],
                                      'pekerjaan': x['pekerjaan'],
                                      'lama pengalaman': x['lama pengalaman'],
                                      'aset': x['aset'],
                                      'ide_bisnis': x['ide_bisnis'],
                                      'label': tf.as_string(x['label'])})

ide_bisnis

<_MapDataset element_spec={'hobi': TensorSpec(shape=(), dtype=tf.string, name=None), 'pekerjaan': TensorSpec(shape=(), dtype=tf.string, name=None), 'lama pengalaman': TensorSpec(shape=(), dtype=tf.string, name=None), 'aset': TensorSpec(shape=(), dtype=tf.string, name=None), 'ide_bisnis': TensorSpec(shape=(), dtype=tf.string, name=None), 'label': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [8]:
tf.random.set_seed(42)
shuffled = ide_bisnis.shuffle(106, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(106*0.8))
test = shuffled.skip(int(106*0.8)).take(106-(int(106*0.8)))

In [9]:
hobi = ide_bisnis.batch(4).map(lambda x: x['hobi'])
pekerjaan = ide_bisnis.batch(4).map(lambda x: x['pekerjaan'])
lama_pengalaman = ide_bisnis.batch(4).map(lambda x: x['lama pengalaman'])
aset = ide_bisnis.batch(4).map(lambda x: x['aset'])

unique_hobi = np.unique(np.concatenate(list(hobi)))
unique_pekerjaan = np.unique(np.concatenate(list(pekerjaan)))
unique_lama_pengalaman = np.unique(np.concatenate(list(lama_pengalaman)))
unique_aset = np.unique(np.concatenate(list(aset)))

In [10]:
class RankingModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
        embedding_dimension = 32
        max_tokens = 10_000
        # Compute embeddings for users.
        self.hobi_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_hobi, mask_token=None),
          tf.keras.layers.Embedding(len(unique_hobi) + 1, embedding_dimension)
        ])
        
        self.pekerjaan_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_pekerjaan, mask_token=None),
          tf.keras.layers.Embedding(len(unique_pekerjaan) + 1, embedding_dimension)
        ])
        
        self.lama_pengalaman_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_lama_pengalaman, mask_token=None),
          tf.keras.layers.Embedding(len(unique_lama_pengalaman) + 1, embedding_dimension)
        ])
            
        self.aset_embeddings = tf.keras.Sequential([
          tf.keras.layers.StringLookup(
            vocabulary=unique_aset, mask_token=None),
          tf.keras.layers.Embedding(len(unique_aset) + 1, embedding_dimension)
        ])

        # Compute embeddings for movies.
        self.ide_bisnis_text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=max_tokens)
        self.ide_bisnis_embeddings = tf.keras.Sequential([
            self.ide_bisnis_text_vectorizer,
            tf.keras.layers.Embedding(max_tokens, embedding_dimension),
            tf.keras.layers.GlobalAveragePooling1D()
        ])
        self.ide_bisnis_text_vectorizer.adapt(ide_bisnis.map(lambda x: x['ide_bisnis']))

        # Compute predictions.
        self.similarity = tf.keras.Sequential([
          # Learn multiple dense layers.
          tf.keras.layers.Dense(256, activation="relu"),
          tf.keras.layers.Dense(64, activation="relu"),
          # Make rating predictions in the final layer.
          tf.keras.layers.Dense(1)
      ])

    def call(self, inputs):

        hobi, pekerjaan, lama_pengalaman, aset, ide_bisnis = inputs

        hobi_embedding = self.hobi_embeddings(hobi)
        pekerjaan_embedding = self.pekerjaan_embeddings(pekerjaan)
        lama_pengalaman_embedding = self.lama_pengalaman_embeddings(lama_pengalaman)
        aset_embedding = self.aset_embeddings(aset)
        user_embedding = tf.concat([hobi_embedding, pekerjaan_embedding, lama_pengalaman_embedding, aset_embedding], axis=1)
        ide_embedding = self.ide_bisnis_embeddings(ide_bisnis)
        
        return self.similarity(tf.concat([user_embedding, ide_embedding], axis=1))
    

In [11]:
RankingModel()((["memotret"], ["guru"], ["1 Sampai 3 Tahun"],["rendah"], ['Buku Panduan Fotografi untuk Pemula']))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.00360916]], dtype=float32)>

In [12]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [13]:
class IdeBisnisModel(tfrs.models.Model):

    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingModel()
        self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )

    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model(
            (features["hobi"], features["pekerjaan"], features["lama pengalaman"], features["aset"], features['ide_bisnis']))

    def compute_loss(self, features: Dict[str, tf.Tensor], training=False) -> tf.Tensor:
        # Extract labels and convert to numeric format
        labels = features.pop("label")
        labels = tf.strings.to_number(labels, tf.float32)  

        rating_predictions = self(features)

    # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

# Fitting and Model Evaluating

In [14]:
model = IdeBisnisModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [15]:
cached_train = train.batch(16).cache()
cached_test = test.batch(10).cache()

In [16]:
model.fit(cached_train, epochs=50)

Epoch 1/50
6/6 [==============================] - 2s 6ms/step - root_mean_squared_error: 0.4346 - loss: 0.1463 - regularization_loss: 0.0000e+00 - total_loss: 0.1463
Epoch 2/50
6/6 [==============================] - 0s 5ms/step - root_mean_squared_error: 0.3550 - loss: 0.0982 - regularization_loss: 0.0000e+00 - total_loss: 0.0982
Epoch 3/50
6/6 [==============================] - 0s 4ms/step - root_mean_squared_error: 0.3514 - loss: 0.0960 - regularization_loss: 0.0000e+00 - total_loss: 0.0960
Epoch 4/50
6/6 [==============================] - 0s 4ms/step - root_mean_squared_error: 0.3487 - loss: 0.0944 - regularization_loss: 0.0000e+00 - total_loss: 0.0944
Epoch 5/50
6/6 [==============================] - 0s 4ms/step - root_mean_squared_error: 0.3465 - loss: 0.0930 - regularization_loss: 0.0000e+00 - total_loss: 0.0930
Epoch 6/50
6/6 [==============================] - 0s 4ms/step - root_mean_squared_error: 0.3446 - loss: 0.0919 - regularization_loss: 0.0000e+00 - total_loss: 0.0919
Epoc

In [17]:
model.evaluate(cached_test, return_dict=True)

3/3 [==============================] - 0s 5ms/step - root_mean_squared_error: 0.4304 - loss: 0.1197 - regularization_loss: 0.0000e+00 - total_loss: 0.1197


{'root_mean_squared_error': 0.4304179847240448,
 'loss': 0.039535921066999435,
 'regularization_loss': 0,
 'total_loss': 0.039535921066999435}

In [18]:
test_similarity = {}
test_ide_bisnis = ["Katering Rumahan", "Jasa Memasak", "Kanal Youtube Memasak", "Kelas Finansial Bisnis "]
for ide in test_ide_bisnis:
    test_similarity[ide] = model({
      "hobi": np.array(["memasak"]),
      "pekerjaan": np.array(["akuntan"]),
      "lama pengalaman": np.array(["Kurang dari 2 Tahun"]),
      "aset": np.array(["rendah"]),
      "ide_bisnis": np.array([ide])
  })

print("Similarity:")
for ide, score in sorted(test_similarity.items(), key=lambda x: x[1], reverse=True):
  print(f"{ide}: {score}")

Similarity:
Katering Rumahan: [[1.1610051]]
Jasa Memasak: [[0.9978322]]
Kanal Youtube Memasak: [[0.91113997]]
Kelas Finansial Bisnis : [[0.7526553]]
